In [2]:
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.layers import Normalization
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Flatten
from keras import layers

2021-08-25 17:34:10.831259: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-25 17:34:10.831399: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
df_merged = pd.read_csv("df_merged.csv")

In [9]:
df_merged["created_at"] = pd.to_datetime(df_merged["created_at"])

In [10]:
df_merged = df_merged.groupby([pd.Grouper(key='created_at',freq='D'), "party"]).agg({
    "replies_count": "sum", 
    "retweets_count": "sum",
    "likes_count": "sum",
    "avg_len_of_tweet": "mean" ,
    "share_of_tweets" :"mean",
    "share_unique_users": "mean",
    "weighted_sentiment" : "mean",
    "share_of_positive_tweets": "mean",
    "share_of_negative_tweets" : "mean",
    "Polls": "mean"}) 

In [11]:
df_merged

replies_count  retweets_count  likes_count  \
created_at party                                                
2021-05-26 AFD               0.0             0.0          0.0   
           CDU               0.0             0.0          0.0   
           FDP               0.0             0.0          0.0   
           GRUENE            0.0             0.0          0.0   
           LINKE             0.0             0.0          0.0   
...                          ...             ...          ...   
2021-08-24 AFD              14.0             6.0         34.0   
           FDP               6.0             3.0         19.0   
           GRUENE           15.0             5.0         23.0   
           LINKE             5.0             1.0          5.0   
           SPD              31.0            18.0         73.0   

                   avg_len_of_tweet  share_of_tweets  share_unique_users  \
created_at party                                                           
2021-05-26 AFD                  NaN              NaN                 NaN   
           CDU                  NaN              NaN                 NaN   
           FDP                  NaN              NaN                 NaN   
           GRUENE               NaN              NaN                 NaN   
           LINKE                NaN              NaN                 NaN   
...                             ...              ...                 ...   
2021-08-24 AFD           191.638298         0.137427            0.702128   
           FDP           170.000000         0.166667            0.929825   
           GRUENE        195.392857         0.245614            0.821429   
           LINKE         150.962963         0.078947            0.777778   
           SPD           201.070866         0.371345            0.740157   

                   weighted_sentiment  share_of_positive_tweets  \
created_at party                                                  
2021-05-26 AFD                    NaN                       NaN   
           CDU                    NaN                       NaN   
           FDP                    NaN                       NaN   
           GRUENE                 NaN                       NaN   
           LINKE                  NaN                       NaN   
...                               ...                       ...   
2021-08-24 AFD                    NaN                       NaN   
           FDP                    NaN                       NaN   
           GRUENE                 NaN                       NaN   
           LINKE                  NaN                       NaN   
           SPD                    NaN                       NaN   

                   share_of_negative_tweets  Polls  
created_at party                                    
2021-05-26 AFD                          NaN  14.38  
           CDU                          NaN  25.50  
           FDP                          NaN  11.62  
           GRUENE                       NaN  20.25  
           LINKE                        NaN   7.88  
...                                     ...    ...  
2021-08-24 AFD                          NaN    NaN  
           FDP                          NaN    NaN  
           GRUENE                       NaN    NaN  
           LINKE                        NaN    NaN  
           SPD                          NaN    NaN  

[634 rows x 10 columns]

In [12]:
model = Sequential()
model.add(layers.LSTM(units=20, activation='tanh', return_sequences=True))
model.add(layers.LSTM(units=20, activation='tanh', return_sequences=False))
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(6, activation='softmax'))
model.build((93, 6, 7))
model.summary()


2021-08-25 17:37:01.920939: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-08-25 17:37:01.921008: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-08-25 17:37:01.921030: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LAPTOP-69AIGBNI): /proc/driver/nvidia/version does not exist
2021-08-25 17:37:01.921437: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (93, 6, 20)               2240      
_________________________________________________________________
lstm_1 (LSTM)                (93, 20)                  3280      
_________________________________________________________________
dense (Dense)                (93, 50)                  1050      
_________________________________________________________________
dropout (Dropout)            (93, 50)                  0         
_________________________________________________________________
dense_1 (Dense)              (93, 6)                   306       
Total params: 6,876
Trainable params: 6,876
Non-trainable params: 0
_________________________________________________________________


In [14]:


model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model.fit(X_train, y_train, 
              epochs=1, 
              batch_size=1, 
              verbose=1, 
              callbacks = [es],
              validation_split=0.2)

model.evaluate(X_test, y_test)

NameError: name 'X_train' is not defined

# MAKE DATA STATIONARY?//???